In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from math import *
import keras

In [ ]:
df = pd.read_csv('data/sims.csv')
df.head(40)

In [ ]:
test = df.loc[(df['set']==1) & (df['ef']==200) & (df['track']==1)]

fig,axs = plt.subplots(1,2)
axs[0].plot(test['x'],test['y'],'ro-')

axs[0].axhline(y=0, color='k')
axs[0].axvline(x=0, color='k')

axs[1].plot(test['slice'],test['speed'])

test.head(50)

In [ ]:
def get_xy(dist,direct,prev_x,prev_y,prev_heading):
    candidates = []
    
    theta1 = acos(direct) #directedness in [0,pi]
    theta2 = 2*acos(-1) - theta1 #directedness in [pi,2*pi]
    
    slope1 = sin(theta1)/cos(theta1) #slope of line of theta1
    slope2 = sin(theta2)/cos(theta2) #slope of line of theta2
    
    dist1 = abs(slope1*prev_x - prev_y)/sqrt(slope1**2+1) #distance of prev point to line 1
    dist2 = abs(slope2*prev_x - prev_y)/sqrt(slope2**2+1) #distance of prev point to line 2
    
    if dist1<dist: #if the distance to line 1 is less than our predicted distance, next point may lie on that line
        #print('line 1')
        closest_x = -1*(-prev_x - slope1*prev_y) / (slope1**2 + 1) #x-value of point on line 1 closest to prev point
        closest_y = slope1*(prev_x + slope1*prev_y) / (slope1**2 + 1) #y-value of point on line 1 closest to prev point 
        #print('closest: ({}, {})'.format(closest_x,closest_y))
        
        r = sqrt(closest_x**2 + closest_y**2) #distance of that closest point from the origin
        
        delta_r = sqrt(dist**2 - dist1**2) #distance of candidate points from that closest point
        
        candx = (r-delta_r)*cos(theta1)
        candy = (r-delta_r)*sin(theta1)
        # handle situation where we get the angle right but in the opposite quadrant
        if closest_x != 0 and closest_y != 0:
            if candx/abs(candx) != closest_x/abs(closest_x) and candy/abs(candy) != closest_y/abs(closest_y):
                candx = -candx
                candy = -candy
        candidates.append((candx,candy)) #first candidate point on line 1
        
        candx = (r+delta_r)*cos(theta1)
        candy = (r+delta_r)*sin(theta1)
        # handle situation where we get the angle right but in the opposite quadrant
        if closest_x != 0 and closest_y != 0:
            if candx/abs(candx) != closest_x/abs(closest_x) and candy/abs(candy) != closest_y/abs(closest_y):
                candx = -candx
                candy = -candy
        candidates.append((candx,candy)) #second candidate point on line 1
        
    if dist2<dist: #if the distance to line 2 is less than our predicted distance, next point may lie on that line
        #print('line 2')
        closest_x = -1*(-prev_x - slope2*prev_y) / (slope2**2 + 1) #x-value of point on line 2 closest to prev point
        closest_y = slope2*(prev_x + slope2*prev_y) / (slope2**2 + 1) #y-value of point on line 2 closest to prev point
        
        #plt.plot([prev_x,closest_x], [prev_y,closest_y], 'b')
        
        r = sqrt(closest_x**2 + closest_y**2) #distance of that closest point from the origin

        delta_r = sqrt(dist**2 - dist2**2) #distance of candidate points from that closest point
        
        candx = (r-delta_r)*cos(theta2)
        candy = (r-delta_r)*sin(theta2)
        # handle situation where we get the angle right but in the opposite quadrant
        if closest_x != 0 and closest_y != 0:
            if candx/abs(candx) != closest_x/abs(closest_x) and candy/abs(candy) != closest_y/abs(closest_y):
                candx = -candx
                candy = -candy
        candidates.append((candx,candy)) #first candidate point on line 2
        
        candx = (r+delta_r)*cos(theta2)
        candy = (r+delta_r)*sin(theta2)
        # handle situation where we get the angle right but in the opposite quadrant
        if closest_x != 0 and closest_y != 0:
            if candx/abs(candx) != closest_x/abs(closest_x) and candy/abs(candy) != closest_y/abs(closest_y):
                candx = -candx
                candy = -candy
        candidates.append((candx,candy)) #second candidate point on line 2
        
        #plt.plot([candidates[0][0], candidates[1][0]], [candidates[0][1], candidates[1][1]], 'y')

    if dist1>dist and dist2>dist:
        if dist1<dist2:
            closest_x = -(-prev_x - slope1*prev_y) / (slope1**2+1) #x-value of point on line 1 closest to prev point
            closest_y = slope1*(prev_x + slope1*prev_y) / (slope1**2+1) #y-value of point on line 1 closest to prev point
            return (closest_x,closest_y)
        else:
            closest_x = -(-prev_x - slope2*prev_y) / (slope2**2+1) #x-value of point on line 2 closest to prev point
            closest_y = slope2*(prev_x + slope2*prev_y) / (slope2**2+1) #y-value of point on line 2 closest to prev point
            return (closest_x,closest_y)
    
    #print('candidates: {}'.format(candidates))
    #choose the candidate point that has the closest heading to the previous heading
    best_cand = (0,0) #bext (x,y) candidate so far
    heading_diff = float('inf') #difference between new heading and prev_heading
    for cand in candidates: #consider each candidate point
        #plt.plot(cand[0],cand[1],'ro')
        #plt.plot([0,cand[0]], [0,cand[1]])
        #plt.plot([prev_x,cand[0]], [prev_y,cand[1]], 'g')
        
        #compute heading from prev point to this candidate
        deltax = cand[0] - prev_x
        deltay = cand[1] - prev_y
        heading = atan(deltay/deltax)
        if deltax<0:
            heading = heading + acos(-1)
        elif deltay<0:
            heading = heading + 2*acos(-1)
        
        #if this is closest heading so far, update
        if abs(heading - prev_heading) < heading_diff:
            heading_diff = abs(heading - prev_heading)
            best_cand = cand
    
    return best_cand

In [ ]:
dist = test.loc[test['slice']==21, 'speed'].values[0] * 5
direct = test.loc[test['slice']==21, 'cum_dir'].values[0]
prev_x = test.loc[test['slice']==20, 'x'].values[0]
prev_y = test.loc[test['slice']==20, 'y'].values[0]

deltax = test.loc[test['slice']==20, 'x'].values[0] - test.loc[test['slice']==19, 'x'].values[0]
deltay = test.loc[test['slice']==20, 'y'].values[0] - test.loc[test['slice']==19, 'y'].values[0]
prev_heading = atan(deltay/deltax)
if deltax<0:
    prev_heading = prev_heading + acos(-1)
elif deltay<0:
    prev_heading = prev_heading + 2*acos(-1)

xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)

print(xy)

x = test.loc[test['slice']==21, 'x'].values[0]
y = test.loc[test['slice']==21, 'y'].values[0]
plt.plot(x,y,'g*')
plt.plot(xy[0],xy[1],'bo')

In [ ]:
xs = []
ys = []

dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
direct = test.loc[test['slice']==21, 'cum_dir'].values[0]
prev_x = test.loc[test['slice']==20, 'x'].values[0]
prev_y = test.loc[test['slice']==20, 'y'].values[0]

deltax = test.loc[test['slice']==20, 'x'].values[0] - test.loc[test['slice']==19, 'x'].values[0]
deltay = test.loc[test['slice']==20, 'y'].values[0] - test.loc[test['slice']==19, 'y'].values[0]
prev_heading = atan(deltay/deltax)
if deltax<0:
    prev_heading = prev_heading + acos(-1)
elif deltay<0:
    prev_heading = prev_heading + 2*acos(-1)

xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
xs.append(xy[0])
ys.append(xy[1])
print(xy)


dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
direct = test.loc[test['slice']==22, 'cum_dir'].values[0]
prev_x = test.loc[test['slice']==21, 'x'].values[0]
prev_y = test.loc[test['slice']==21, 'y'].values[0]

deltax = xs[0] - test.loc[test['slice']==20, 'x'].values[0]
deltay = ys[0] - test.loc[test['slice']==20, 'y'].values[0]
prev_heading = atan(deltay/deltax)
if deltax<0:
    prev_heading = prev_heading + acos(-1)
elif deltay<0:
    prev_heading = prev_heading + 2*acos(-1)

xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
xs.append(xy[0])
ys.append(xy[1])
print(xy)


for i in range(2,17):
    dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
    direct = test.loc[test['slice']==i+21, 'cum_dir'].values[0]
    prev_x = test.loc[test['slice']==i+20, 'x'].values[0]
    prev_y = test.loc[test['slice']==i+20, 'y'].values[0]

    deltax = xs[i-1] - xs[i-2]
    deltay = ys[i-1] - ys[i-2]
    prev_heading = atan(deltay/deltax)
    if deltax<0:
        prev_heading = prev_heading + acos(-1)
    elif deltay<0:
        prev_heading = prev_heading + 2*acos(-1)
    
    xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
    xs.append(xy[0])
    ys.append(xy[1])
    print(xy)

plt.plot(test['x'],test['y'])
plt.plot(xs,ys,'r')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.savefig('trajectory')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        #direct = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
        direct = test.loc[test['slice']==i+1, 'sim0'].values[0]
        #prev_x = test.loc[test['slice']==i, 'x'].values[0]
        #prev_y = test.loc[test['slice']==i, 'y'].values[0]
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)

        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        xs.append(xy[0])
        ys.append(xy[1])
        print(xy)

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(20,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)
    
    groundxs = []
    groundys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        groundxs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        groundys.append(y)

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        direct = test.loc[test['slice']==i+1, 'sim0'].values[0]
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        grounddirect = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
        groundprev_x = test.loc[test['slice']==i, 'x'].values[0]
        groundprev_y = test.loc[test['slice']==i, 'y'].values[0]
        
        grounddeltax = groundxs[i-1] - groundxs[i-2]
        grounddeltay = groundys[i-1] - groundys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)
                
        if grounddeltax==0: #if deltax is zero, heading determined by sign of y
            if grounddeltay>0:
                groundprev_heading = acos(-1)/2 #pi/2
            else:
                groundprev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            groundprev_heading = atan(grounddeltay/grounddeltax)
            if grounddeltax<0:
                groundprev_heading = groundprev_heading + acos(-1)
            elif grounddeltay<0:
                groundprev_heading = groundprev_heading + 2*acos(-1)

        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        xs.append(xy[0])
        ys.append(xy[1])
        print(xy)
        
        groundxy = get_xy(dist,grounddirect,groundprev_x,groundprev_y,groundprev_heading)
        groundxs.append(groundxy[0])
        groundys.append(groundxy[1])

    axs[0].plot(test['x'],test['y'])
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    
    axs[1].plot(groundxs,groundys)
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
    axs[2].plot(xs,ys)
    axs[2].axhline(y=0, color='k')
    axs[2].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Ground Truth Directedness')
axs[2].set_title('Derived Positions Using Simulated Directedness')

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot()
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    groundxs = []
    groundys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        groundxs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        groundys.append(y)

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        direct = test.loc[test['slice']==i+1, 'sim0'].values[0]
        
        grounddirect = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
        groundprev_x = test.loc[test['slice']==i, 'x'].values[0]
        groundprev_y = test.loc[test['slice']==i, 'y'].values[0]
        
        grounddeltax = groundxs[i-1] - groundxs[i-2]
        grounddeltay = groundys[i-1] - groundys[i-2]
                
        if grounddeltax==0: #if deltax is zero, heading determined by sign of y
            if grounddeltay>0:
                groundprev_heading = acos(-1)/2 #pi/2
            else:
                groundprev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            groundprev_heading = atan(grounddeltay/grounddeltax)
            if grounddeltax<0:
                groundprev_heading = groundprev_heading + acos(-1)
            elif grounddeltay<0:
                groundprev_heading = groundprev_heading + 2*acos(-1)
        
        groundxy = get_xy(dist,grounddirect,groundprev_x,groundprev_y,groundprev_heading)
        groundxs.append(groundxy[0])
        groundys.append(groundxy[1])
    
    ax.plot(groundxs,groundys)
    ax.axhline(y=0, color='k')
    ax.axvline(x=0, color='k')
    
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xticks([])
ax.set_xticklabels([])

ax.set_yticks([])
ax.set_yticklabels([])

fig.savefig('reconstructed_200mVmm_set2.pdf')

# Noisy Trajectories

### Noise on (x,y) position

In [ ]:
#mean and std dev of normal dist for noise
mu, sigma = 0, 5

fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)

    for i in range(20,37):
        #dist_mu = np.mean(test.loc[test['slice']<=20, 'speed'].values[0] * 5)
        #dist_sigma = np.std(test.loc[test['slice']<=20, 'speed'].values[0] * 5)
        #dist = np.random.normal(dist_mu, dist_sigma)
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        #direct = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
        direct = test.loc[test['slice']==i+1, 'sim0'].values[0]
        #prev_x = test.loc[test['slice']==i, 'x'].values[0]
        #prev_y = test.loc[test['slice']==i, 'y'].values[0]
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)

        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        xs.append(xy[0] + np.random.normal(mu, sigma))
        ys.append(xy[1] + np.random.normal(mu, sigma))
        print(xy)

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

### Noise on Heading

In [ ]:
#mean and std dev of normal dist for noise
mu, sigma = 0, 2*np.pi/3

fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        #direct = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
        direct = test.loc[test['slice']==i+1, 'sim0'].values[0]
        #prev_x = test.loc[test['slice']==i, 'x'].values[0]
        #prev_y = test.loc[test['slice']==i, 'y'].values[0]
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)
        
        prev_heading += np.random.normal(mu, sigma)

        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        xs.append(xy[0])
        ys.append(xy[1])
        print(xy)

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

### Noise on directedness used as simulation inputs

In [ ]:
# Load CNCC prediction model
print('Loading model...')
model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded.')

#mean and std dev of normal dist for noise
mu, sigma = 0, 0.01

fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    dirs = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)
    for cumdir in test.loc[test['slice']<=20,'cum_dir']:
        dirs.append(cumdir)
        
    lookback_ef = np.array([0.2 for i in range(20)]).reshape((20,1))

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        
        #get directedness via simulation
        lookback_dir = np.array(dirs[-20:]).reshape((20,1))
        lookback = np.hstack((lookback_ef,lookback_dir)).reshape((1,20,2))
        direct = model.predict(lookback)[0,0] + np.random.normal(mu, sigma)
        if direct < -1:
            direct = -1
        elif direct > 1:
            direct = 1
        dirs.append(direct)
        
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)

        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        xs.append(xy[0])
        ys.append(xy[1])
        print(xy)

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

### Noise on (x,y) used for simulation inputs

In [ ]:
# Load CNCC prediction model
print('Loading model...')
model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded.')

#mean and std dev of normal dist for noise
mu, sigma = 0, 15

fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==2) & (df['ef']==200) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    dirs = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)
    for cumdir in test.loc[test['slice']<=20,'cum_dir']:
        dirs.append(cumdir)
        
    lookback_ef = np.array([0.2 for i in range(20)]).reshape((20,1))

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        
        #get directedness via simulation
        lookback_dir = np.array(dirs[-20:]).reshape((20,1))
        lookback = np.hstack((lookback_ef,lookback_dir)).reshape((1,20,2))
        direct = model.predict(lookback)[0,0]
        if direct < -1:
            direct = -1
        elif direct > 1:
            direct = 1
        
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)
        
        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        
        new_x = xy[0] + np.random.normal(mu, sigma)
        new_y = xy[1] + np.random.normal(mu, sigma)
        new_dir = cos(atan(new_y/new_x))
        if new_x < 0:
            new_dir = -new_dir
        
        xs.append(new_x)
        ys.append(new_y)
        dirs.append(new_dir)
        print(xy)

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

# Rose Plots

In [ ]:
ef = 100
setnum = 2

df['theta'] = np.arccos(df['cum_dir'])
final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]
print(final_dirs[['track','cum_dir','theta']])

props = []
for i in range(6):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = len(final_dirs.loc[(final_dirs['theta']>start_theta)&(final_dirs['theta']<=end_theta)])
    print(cnt)
    props.append(cnt/len(final_dirs))

fig = plt.figure()
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(180)
ax.set_xticks([i*np.pi/6 for i in range(7)])
ax.set_ylim(0,1)

ax.bar([i*np.pi/6+np.pi/12 for i in range(6)],props,width=np.pi/6)

In [ ]:
ef = 100
setnum = 2

df['sim_theta'] = np.arccos(df['sim0'])
final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

props = []
for i in range(6):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = len(final_dirs.loc[(final_dirs['sim_theta']>start_theta)&(final_dirs['sim_theta']<=end_theta)])
    print(cnt)
    props.append(cnt/len(final_dirs))

fig = plt.figure()
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(180)
ax.set_xticks([i*np.pi/6 for i in range(7)])
ax.set_ylim(0,1)

ax.bar([i*np.pi/6+np.pi/12 for i in range(6)],props,width=np.pi/6)

## Rose Plots of Trajectory Plots

In [ ]:
slice_cutoff = 20
relevant_dirs = []
ef = 100
setnum = 2

# Load CNCC prediction model
print('Loading model...')
model = keras.models.load_model('models/cncc_lstm.h5')
print('Model loaded.')

#mean and std dev of normal dist for noise
mu, sigma = 0, 10

fig, axs = plt.subplots(1,2,figsize=(15,5))
#loop through cells in this experiment
for track in range(1,51):
    test = df.loc[(df['set']==setnum) & (df['ef']==ef) & (df['track']==track)] #get cell
    
    xs = []
    ys = []
    dirs = []
    
    for x in test.loc[test['slice']<=20,'x']:
        xs.append(x)
    for y in test.loc[test['slice']<=20,'y']:
        ys.append(y)
    for cumdir in test.loc[test['slice']<=20,'cum_dir']:
        dirs.append(cumdir)
        
    lookback_ef = np.array([0.2 for i in range(20)]).reshape((20,1))

    for i in range(20,37):
        dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
        
        #get directedness via simulation
        lookback_dir = np.array(dirs[-20:]).reshape((20,1))
        lookback = np.hstack((lookback_ef,lookback_dir)).reshape((1,20,2))
        direct = model.predict(lookback)[0,0]
        if direct < -1:
            direct = -1
        elif direct > 1:
            direct = 1
        
        prev_x = xs[-1]
        prev_y = ys[-1]

        deltax = xs[i-1] - xs[i-2]
        deltay = ys[i-1] - ys[i-2]
        
        if deltax==0: #if deltax is zero, heading determined by sign of y
            if deltay>0:
                prev_heading = acos(-1)/2 #pi/2
            else:
                prev_heading = 3*acos(-1)/2 #3*pi/2
        else:
            prev_heading = atan(deltay/deltax)
            if deltax<0:
                prev_heading = prev_heading + acos(-1)
            elif deltay<0:
                prev_heading = prev_heading + 2*acos(-1)
        
        xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
        
        new_x = xy[0] + np.random.normal(mu, sigma)
        new_y = xy[1] + np.random.normal(mu, sigma)
        new_dir = cos(atan(new_y/new_x))
        if new_x < 0:
            new_dir = -new_dir
        
        xs.append(new_x)
        ys.append(new_y)
        dirs.append(new_dir)
        print(xy)
        
        theta = atan2(new_y,new_x)
        if theta<0:
            theta += 2*np.pi
        df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['track']==track)&(df['slice']==i), 'sim_theta'] = theta

    axs[0].plot(test['x'],test['y'])
    axs[1].plot(xs,ys)
    axs[0].axhline(y=0, color='k')
    axs[0].axvline(x=0, color='k')
    axs[1].axhline(y=0, color='k')
    axs[1].axvline(x=0, color='k')
    
axs[0].set_title('Ground Truth Positions')
axs[1].set_title('Derived Positions Using Simulated Directedness and Constant Speed')

print(len(df.loc[df['sim_theta']>0]))

In [ ]:
ef = 100
setnum = 2

final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

props = []
for i in range(12):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = len(final_dirs.loc[(final_dirs['sim_theta']>start_theta)&(final_dirs['sim_theta']<=end_theta)])
    print(cnt)
    props.append(cnt/len(final_dirs))
print('total: {}'.format(len(final_dirs)))

fig = plt.figure()
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(360)
ax.set_xticks([i*np.pi/6 for i in range(13)])
ax.set_ylim(0,0.5)

ax.bar([i*np.pi/6+np.pi/12 for i in range(12)],props,width=np.pi/6)

In [ ]:
#GROUND TRUTH

ef = 200
setnum = 2

df['theta'] = np.arctan2(df['y'],df['x'])
df.loc[df['theta']<0, 'theta'] = df.loc[df['theta']<0, 'theta'] + 2*np.pi

final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

props = []
for i in range(12):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = len(final_dirs.loc[(final_dirs['theta']>start_theta)&(final_dirs['theta']<=end_theta)])
    print('count: {}, prop: {}'.format(cnt,cnt/len(final_dirs)))
    props.append(cnt/len(final_dirs))
print('total: {}'.format(len(final_dirs)))

fig = plt.figure(figsize=(10,10), dpi=240)
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(360)
ax.set_xticks([i*np.pi/6 for i in range(12)])
ax.set_ylim(0,0.5)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
ax.set_title('{}mV/mm Ground Truth'.format(ef), fontsize=20)

ax.bar([i*np.pi/6+np.pi/12 for i in range(12)],props,width=np.pi/6)

fig.savefig('figures/{}mV_ground_rose.pdf'.format(ef))

## Simulation Rose Plot Percentages for All Models

In [ ]:
slice_cutoff = 20
relevant_dirs = []
ef = 200
setnum = 2

for mod in range(50):
    # Load CNCC prediction model
    print('Loading model {}...'.format(mod))
    model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(mod))
    print('Model loaded.')

    #mean and std dev of normal dist for noise
    mu, sigma = 0, 10

    fig, axs = plt.subplots(1,2,figsize=(15,5))
    #loop through cells in this experiment
    for track in range(1,51):
        test = df.loc[(df['set']==setnum) & (df['ef']==ef) & (df['track']==track)] #get cell

        xs = []
        ys = []
        dirs = []

        for x in test.loc[test['slice']<=20,'x']:
            xs.append(x)
        for y in test.loc[test['slice']<=20,'y']:
            ys.append(y)
        for cumdir in test.loc[test['slice']<=20,'cum_dir']:
            dirs.append(cumdir)

        lookback_ef = np.array([0.2 for i in range(20)]).reshape((20,1))

        for i in range(20,37):
            dist = test.loc[test['slice']==20, 'speed'].values[0] * 5

            #get directedness via simulation
            lookback_dir = np.array(dirs[-20:]).reshape((20,1))
            lookback = np.hstack((lookback_ef,lookback_dir)).reshape((1,20,2))
            direct = model.predict(lookback)[0,0]
            if direct < -1:
                direct = -1
            elif direct > 1:
                direct = 1

            prev_x = xs[-1]
            prev_y = ys[-1]

            deltax = xs[i-1] - xs[i-2]
            deltay = ys[i-1] - ys[i-2]

            if deltax==0: #if deltax is zero, heading determined by sign of y
                if deltay>0:
                    prev_heading = acos(-1)/2 #pi/2
                else:
                    prev_heading = 3*acos(-1)/2 #3*pi/2
            else:
                prev_heading = atan(deltay/deltax)
                if deltax<0:
                    prev_heading = prev_heading + acos(-1)
                elif deltay<0:
                    prev_heading = prev_heading + 2*acos(-1)

            xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)

            new_x = xy[0] + np.random.normal(mu, sigma)
            new_y = xy[1] + np.random.normal(mu, sigma)
            new_dir = cos(atan(new_y/new_x))
            if new_x < 0:
                new_dir = -new_dir

            xs.append(new_x)
            ys.append(new_y)
            dirs.append(new_dir)
            #print(xy)

            theta = atan2(new_y,new_x)
            if theta<0:
                theta += 2*np.pi
            df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['track']==track)&(df['slice']==i), 'sim_theta{}'.format(mod)] = theta

In [ ]:
final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

sim_props = []
for i in range(12):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = 0
    total = 0
    for mod in range(50):
        cnt += len(final_dirs.loc[(final_dirs['sim_theta{}'.format(mod)]>start_theta)&(final_dirs['sim_theta{}'.format(mod)]<=end_theta)])
        total += len(final_dirs)
    print('Start: {}, End: {}, count: {}, total: {}, prop: {}'.format(start_theta,end_theta,cnt,total,cnt/total))
    sim_props.append(cnt/total)
print('total: {}'.format(len(final_dirs)))
              
fig = plt.figure(figsize=(10,10), dpi=240)
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(360)
ax.set_xticks([i*np.pi/6 for i in range(12)])
ax.set_ylim(0,0.5)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
ax.set_title('{}mV/mm Noisy Simulation'.format(ef), fontsize=20)

ax.bar([i*np.pi/6+np.pi/12 for i in range(12)],sim_props,width=np.pi/6)

fig.savefig('figures/{}mV_noisy_rose.pdf'.format(ef))

### Simulation Without Noise

In [ ]:
ef = 75
setnum = 2

#loop through the models
for mod in range(50):
    #loop through cells in this experiment
    for track in range(1,51):
        test = df.loc[(df['set']==2) & (df['ef']==ef) & (df['track']==track)] #get cell

        xs = []
        ys = []

        for x in test.loc[test['slice']<=20,'x']:
            xs.append(x)
        for y in test.loc[test['slice']<=20,'y']:
            ys.append(y)

        groundxs = []
        groundys = []

        for x in test.loc[test['slice']<=20,'x']:
            groundxs.append(x)
        for y in test.loc[test['slice']<=20,'y']:
            groundys.append(y)

        for i in range(20,37):
            dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
            direct = test.loc[test['slice']==i+1, 'sim{}'.format(mod)].values[0]
            prev_x = xs[-1]
            prev_y = ys[-1]

            deltax = xs[i-1] - xs[i-2]
            deltay = ys[i-1] - ys[i-2]

            grounddirect = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
            groundprev_x = test.loc[test['slice']==i, 'x'].values[0]
            groundprev_y = test.loc[test['slice']==i, 'y'].values[0]

            grounddeltax = groundxs[i-1] - groundxs[i-2]
            grounddeltay = groundys[i-1] - groundys[i-2]

            if deltax==0: #if deltax is zero, heading determined by sign of y
                if deltay>0:
                    prev_heading = acos(-1)/2 #pi/2
                else:
                    prev_heading = 3*acos(-1)/2 #3*pi/2
            else:
                prev_heading = atan(deltay/deltax)
                if deltax<0:
                    prev_heading = prev_heading + acos(-1)
                elif deltay<0:
                    prev_heading = prev_heading + 2*acos(-1)

            if grounddeltax==0: #if deltax is zero, heading determined by sign of y
                if grounddeltay>0:
                    groundprev_heading = acos(-1)/2 #pi/2
                else:
                    groundprev_heading = 3*acos(-1)/2 #3*pi/2
            else:
                groundprev_heading = atan(grounddeltay/grounddeltax)
                if grounddeltax<0:
                    groundprev_heading = groundprev_heading + acos(-1)
                elif grounddeltay<0:
                    groundprev_heading = groundprev_heading + 2*acos(-1)

            xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
            xs.append(xy[0])
            ys.append(xy[1])

            groundxy = get_xy(dist,grounddirect,groundprev_x,groundprev_y,groundprev_heading)
            groundxs.append(groundxy[0])
            groundys.append(groundxy[1])
            
            theta = atan2(xy[1],xy[0])
            if theta<0:
                theta += 2*np.pi
            df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['track']==track)&(df['slice']==i), 'sim_theta{}'.format(mod)] = theta

In [ ]:
final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

props = []
for i in range(12):
    start_theta = i*np.pi/6
    end_theta = (i+1)*np.pi/6
    
    cnt = 0
    total = 0
    for mod in range(50):
        cnt += len(final_dirs.loc[(final_dirs['sim_theta{}'.format(mod)]>start_theta)&(final_dirs['sim_theta{}'.format(mod)]<=end_theta)])
        total += len(final_dirs)
    print('Start: {}, End: {}, count: {}, total: {}, prop: {}'.format(start_theta,end_theta,cnt,total,cnt/total))
    props.append(cnt/total)
print('total: {}'.format(len(final_dirs)))
              
fig = plt.figure(figsize=(10,10), dpi=240)
ax = fig.add_subplot(projection='polar')
ax.set_thetamin(0)
ax.set_thetamax(360)
ax.set_xticks([i*np.pi/6 for i in range(12)])
ax.set_ylim(0,0.5)
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=18)
ax.set_title('{}mV/mm Simulation'.format(ef), fontsize=28)

ax.bar([i*np.pi/6+np.pi/12 for i in range(12)],props,width=np.pi/6)

fig.savefig('figures/{}mV_sim_rose.pdf'.format(ef))

## Compare accuracy of noisy to regular sims

In [ ]:
prop_dict = {}
for ef in [0,15,30,50,75,100,200]:
    print('{}mV/mm'.format(ef))
    
    slice_cutoff = 20
    relevant_dirs = []
    setnum = 2

    for mod in range(50):
        # Load CNCC prediction model
        print('Loading model {}...'.format(mod))
        model = keras.models.load_model('models/cncc_lstm/model{}.h5'.format(mod))
        print('Model loaded.')

        #mean and std dev of normal dist for noise
        mu, sigma = 0, 10

        #loop through cells in this experiment
        for track in range(1,51):
            test = df.loc[(df['set']==setnum) & (df['ef']==ef) & (df['track']==track)] #get cell

            xs = []
            ys = []
            dirs = []

            for x in test.loc[test['slice']<=20,'x']:
                xs.append(x)
            for y in test.loc[test['slice']<=20,'y']:
                ys.append(y)
            for cumdir in test.loc[test['slice']<=20,'cum_dir']:
                dirs.append(cumdir)

            lookback_ef = np.array([0.2 for i in range(20)]).reshape((20,1))

            for i in range(20,37):
                dist = test.loc[test['slice']==20, 'speed'].values[0] * 5

                #get directedness via simulation
                lookback_dir = np.array(dirs[-20:]).reshape((20,1))
                lookback = np.hstack((lookback_ef,lookback_dir)).reshape((1,20,2))
                direct = model.predict(lookback)[0,0]
                if direct < -1:
                    direct = -1
                elif direct > 1:
                    direct = 1

                prev_x = xs[-1]
                prev_y = ys[-1]

                deltax = xs[i-1] - xs[i-2]
                deltay = ys[i-1] - ys[i-2]

                if deltax==0: #if deltax is zero, heading determined by sign of y
                    if deltay>0:
                        prev_heading = acos(-1)/2 #pi/2
                    else:
                        prev_heading = 3*acos(-1)/2 #3*pi/2
                else:
                    prev_heading = atan(deltay/deltax)
                    if deltax<0:
                        prev_heading = prev_heading + acos(-1)
                    elif deltay<0:
                        prev_heading = prev_heading + 2*acos(-1)

                xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)

                new_x = xy[0] + np.random.normal(mu, sigma)
                new_y = xy[1] + np.random.normal(mu, sigma)
                new_dir = cos(atan(new_y/new_x))
                if new_x < 0:
                    new_dir = -new_dir

                xs.append(new_x)
                ys.append(new_y)
                dirs.append(new_dir)
                #print(xy)

                theta = atan2(new_y,new_x)
                if theta<0:
                    theta += 2*np.pi
                df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['track']==track)&(df['slice']==i), 'sim_theta{}'.format(mod)] = theta
                
    final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

    sim_props = []
    for i in range(12):
        start_theta = i*np.pi/6
        end_theta = (i+1)*np.pi/6

        cnt = 0
        total = 0
        for mod in range(50):
            cnt += len(final_dirs.loc[(final_dirs['sim_theta{}'.format(mod)]>start_theta)&(final_dirs['sim_theta{}'.format(mod)]<=end_theta)])
            total += len(final_dirs)
        print('Start: {}, End: {}, count: {}, total: {}, prop: {}'.format(start_theta,end_theta,cnt,total,cnt/total))
        sim_props.append(cnt/total)
    print('total: {}'.format(len(final_dirs)))

    prop_dict[ef] = sim_props

print(prop_dict)

In [ ]:
efs = [0,15,30,50,75,100,200]
setnum = 2
noisy_mses = {}
for ef in efs:
    print('{}mV/mm'.format(ef))

    df['theta'] = np.arctan2(df['y'],df['x'])
    df.loc[df['theta']<0, 'theta'] = df.loc[df['theta']<0, 'theta'] + 2*np.pi

    final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

    props = []
    for i in range(12):
        start_theta = i*np.pi/6
        end_theta = (i+1)*np.pi/6

        cnt = len(final_dirs.loc[(final_dirs['theta']>start_theta)&(final_dirs['theta']<=end_theta)])
        print('count: {}, prop: {}'.format(cnt,cnt/len(final_dirs)))
        props.append(cnt/len(final_dirs))
    print('total: {}'.format(len(final_dirs)))
    
    mse = 0
    for i in range(len(props)):
        mse += (props[i] - prop_dict[ef][i])**2
    mse = mse/len(props)
    noisy_mses[ef] = mse
    
print(noisy_mses)

In [ ]:
sim_prop_dict = {}
slice_cutoff = 20
efs = [0,15,30,50,75,100,200]
for ef in efs:
    print('{}mV/mm'.format(ef))
    
    relevant_dirs = []

    #loop through the models
    for mod in range(50):
        print('Model {}'.format(mod))
        
        #loop through cells in this experiment
        for track in range(1,51):
            test = df.loc[(df['set']==2) & (df['ef']==ef) & (df['track']==track)] #get cell

            xs = []
            ys = []

            for x in test.loc[test['slice']<=20,'x']:
                xs.append(x)
            for y in test.loc[test['slice']<=20,'y']:
                ys.append(y)

            groundxs = []
            groundys = []

            for x in test.loc[test['slice']<=20,'x']:
                groundxs.append(x)
            for y in test.loc[test['slice']<=20,'y']:
                groundys.append(y)

            for i in range(20,37):
                dist = test.loc[test['slice']==20, 'speed'].values[0] * 5
                direct = test.loc[test['slice']==i+1, 'sim{}'.format(mod)].values[0]
                prev_x = xs[-1]
                prev_y = ys[-1]

                deltax = xs[i-1] - xs[i-2]
                deltay = ys[i-1] - ys[i-2]

                grounddirect = test.loc[test['slice']==i+1, 'cum_dir'].values[0]
                groundprev_x = test.loc[test['slice']==i, 'x'].values[0]
                groundprev_y = test.loc[test['slice']==i, 'y'].values[0]

                grounddeltax = groundxs[i-1] - groundxs[i-2]
                grounddeltay = groundys[i-1] - groundys[i-2]

                if deltax==0: #if deltax is zero, heading determined by sign of y
                    if deltay>0:
                        prev_heading = acos(-1)/2 #pi/2
                    else:
                        prev_heading = 3*acos(-1)/2 #3*pi/2
                else:
                    prev_heading = atan(deltay/deltax)
                    if deltax<0:
                        prev_heading = prev_heading + acos(-1)
                    elif deltay<0:
                        prev_heading = prev_heading + 2*acos(-1)

                if grounddeltax==0: #if deltax is zero, heading determined by sign of y
                    if grounddeltay>0:
                        groundprev_heading = acos(-1)/2 #pi/2
                    else:
                        groundprev_heading = 3*acos(-1)/2 #3*pi/2
                else:
                    groundprev_heading = atan(grounddeltay/grounddeltax)
                    if grounddeltax<0:
                        groundprev_heading = groundprev_heading + acos(-1)
                    elif grounddeltay<0:
                        groundprev_heading = groundprev_heading + 2*acos(-1)

                xy = get_xy(dist,direct,prev_x,prev_y,prev_heading)
                xs.append(xy[0])
                ys.append(xy[1])

                groundxy = get_xy(dist,grounddirect,groundprev_x,groundprev_y,groundprev_heading)
                groundxs.append(groundxy[0])
                groundys.append(groundxy[1])

                theta = atan2(xy[1],xy[0])
                if theta<0:
                    theta += 2*np.pi
                df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['track']==track)&(df['slice']==i), 'sim_theta{}'.format(mod)] = theta

    final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

    sim_props = []
    for i in range(12):
        start_theta = i*np.pi/6
        end_theta = (i+1)*np.pi/6

        cnt = 0
        total = 0
        for mod in range(50):
            cnt += len(final_dirs.loc[(final_dirs['sim_theta{}'.format(mod)]>start_theta)&(final_dirs['sim_theta{}'.format(mod)]<=end_theta)])
            total += len(final_dirs)
        print('Start: {}, End: {}, count: {}, total: {}, prop: {}'.format(start_theta,end_theta,cnt,total,cnt/total))
        sim_props.append(cnt/total)
    print('total: {}'.format(len(final_dirs)))

    sim_prop_dict[ef] = sim_props

print(prop_dict)

In [ ]:
efs = [0,15,30,50,75,100,200]
setnum = 2
sim_mses = {}
for ef in efs:
    print('{}mV/mm'.format(ef))

    df['theta'] = np.arctan2(df['y'],df['x'])
    df.loc[df['theta']<0, 'theta'] = df.loc[df['theta']<0, 'theta'] + 2*np.pi

    final_dirs = df.loc[(df['ef']==ef)&(df['set']==setnum)&(df['slice']>20)]

    props = []
    for i in range(12):
        start_theta = i*np.pi/6
        end_theta = (i+1)*np.pi/6

        cnt = len(final_dirs.loc[(final_dirs['theta']>start_theta)&(final_dirs['theta']<=end_theta)])
        print('count: {}, prop: {}'.format(cnt,cnt/len(final_dirs)))
        props.append(cnt/len(final_dirs))
    print('total: {}'.format(len(final_dirs)))
    
    mse = 0
    for i in range(len(props)):
        mse += (props[i] - sim_prop_dict[ef][i])**2
    mse = mse/len(props)
    sim_mses[ef] = mse
    
print(sim_mses)

In [ ]:
plt.figure(figsize=(12,8),dpi=120)
plt.bar([2*x-0.4 for x in range(len(sim_mses))], [sqrt(x) for x in list(sim_mses.values())], label='No Noise')
plt.bar([2*x+0.4 for x in range(len(noisy_mses))], [sqrt(x) for x in list(noisy_mses.values())], label='Gaussian Noise')
plt.xticks([2*x for x in range(len(sim_mses))], [str(x)+'mV/mm' for x in sim_mses.keys()], fontsize=14)
plt.yticks(fontsize=14)
plt.title('Rose Plot Error of Simulations',fontsize=24)
plt.xlabel('Simulation',fontsize=18)
plt.ylabel('RMSE of Proportion of Cells in Slice of Rose Plot',fontsize=18)
plt.legend(fontsize=14)
plt.savefig('figures/rose_rmse_bars.pdf')